In [3]:
# Dependencies
from bs4 import BeautifulSoup
import requests
import os

In [4]:
# URL of page to be scraped
url = 'http://quotes.toscrape.com/'

In [5]:
# Retrieve page with the requests module
response = requests.get(url)

In [6]:
# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(response.text, 'html.parser')

In [7]:
# Examine the results, then determine element that contains sought info
print(soup.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   Quotes to Scrape
  </title>
  <link href="/static/bootstrap.min.css" rel="stylesheet"/>
  <link href="/static/main.css" rel="stylesheet"/>
 </head>
 <body>
  <div class="container">
   <div class="row header-box">
    <div class="col-md-8">
     <h1>
      <a href="/" style="text-decoration: none">
       Quotes to Scrape
      </a>
     </h1>
    </div>
    <div class="col-md-4">
     <p>
      <a href="/login">
       Login
      </a>
     </p>
    </div>
   </div>
   <div class="row">
    <div class="col-md-8">
     <div class="quote" itemscope="" itemtype="http://schema.org/CreativeWork">
      <span class="text" itemprop="text">
       “The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”
      </span>
      <span>
       by
       <small class="author" itemprop="author">
        Albert Einstein
       </small>
       <a href="/author/Albert

In [22]:
# results are returned as an iterable list
results = soup.find_all('div', class_="quote")
results

[<div class="quote" itemscope="" itemtype="http://schema.org/CreativeWork">
 <span class="text" itemprop="text">“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”</span>
 <span>by <small class="author" itemprop="author">Albert Einstein</small>
 <a href="/author/Albert-Einstein">(about)</a>
 </span>
 <div class="tags">
             Tags:
             <meta class="keywords" content="change,deep-thoughts,thinking,world" itemprop="keywords"/>
 <a class="tag" href="/tag/change/page/1/">change</a>
 <a class="tag" href="/tag/deep-thoughts/page/1/">deep-thoughts</a>
 <a class="tag" href="/tag/thinking/page/1/">thinking</a>
 <a class="tag" href="/tag/world/page/1/">world</a>
 </div>
 </div>,
 <div class="quote" itemscope="" itemtype="http://schema.org/CreativeWork">
 <span class="text" itemprop="text">“It is our choices, Harry, that show what we truly are, far more than our abilities.”</span>
 <span>by <small class="author" itempr

In [54]:
# Loop through returned results
for result in results:
    # Error handling
    try:
        # Identify and return title of listing
        quote = result.find('span', class_="text").text
        tags = result.find('div', class_="tags").text
        author = result.find('small', class_="author").text
        details = result.find('a').attrs['href']    
        details_url = f'http://quotes.toscrape.com{details}'
        #details = result.find('div', class_="author_details").text
#         # Identify and return price of listing
         #quote = result.a.span.text
#         # Identify and return link to listing
         

#         # Print results only if title, price, and link are available
        if(quote and author and tags and about ):
            print('-------------')
            print(quote)
            print(tags)
            print(author)
            print(details_url)
          
            
    except AttributeError as e:
        print(e)

-------------
“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”

            Tags:
            
change
deep-thoughts
thinking
world

Albert Einstein
http://quotes.toscrape.com/author/Albert-Einstein
-------------
“It is our choices, Harry, that show what we truly are, far more than our abilities.”

            Tags:
            
abilities
choices

J.K. Rowling
http://quotes.toscrape.com/author/J-K-Rowling
-------------
“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”

            Tags:
            
inspirational
life
live
miracle
miracles

Albert Einstein
http://quotes.toscrape.com/author/Albert-Einstein
-------------
“The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.”

            Tags:
            
aliteracy
books
classic
humor

Jane Austen
http://quotes.toscrape.com/author/Jane-Austen

In [43]:
details = soup.find_all('div', class_="")
details

[<div class="container">
 <div class="row header-box">
 <div class="col-md-8">
 <h1>
 <a href="/" style="text-decoration: none">Quotes to Scrape</a>
 </h1>
 </div>
 <div class="col-md-4">
 <p>
 <a href="/login">Login</a>
 </p>
 </div>
 </div>
 <div class="row">
 <div class="col-md-8">
 <div class="quote" itemscope="" itemtype="http://schema.org/CreativeWork">
 <span class="text" itemprop="text">“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”</span>
 <span>by <small class="author" itemprop="author">Albert Einstein</small>
 <a href="/author/Albert-Einstein">(about)</a>
 </span>
 <div class="tags">
             Tags:
             <meta class="keywords" content="change,deep-thoughts,thinking,world" itemprop="keywords"/>
 <a class="tag" href="/tag/change/page/1/">change</a>
 <a class="tag" href="/tag/deep-thoughts/page/1/">deep-thoughts</a>
 <a class="tag" href="/tag/thinking/page/1/">thinking</a>
 <a class="tag" href="/tag

In [42]:
# Loop through returned results
for detail in details:
    # Error handling
    try:
        # Identify and return title of listing
        born = result.find('span', class_="author-born-date").text
        description = result.find('div', class_="author-description").text
 
        if(quote and author and tags and about ):
            print('-------------')
            print(born)
            print(description)         
            
    except AttributeError as e:
        print(e)